In [1]:
import cv2
import numpy as np
#import scipy.misc
#import os
#import time
#os.chdir("/home/mckc/Imagedb/")
#import uuid
import dlib

detector = dlib.get_frontal_face_detector()
video_capture = cv2.VideoCapture(0)

In [2]:
import cPickle
def standard(X):
    return (X - X.mean())/X.max()

def Pre_Process(face):
    from skimage.transform import resize
    X = standard(resize(face,(96,96))).reshape(-1,1,96,96)
    X_normal = X.reshape(-1,9216)
    return X,X_normal

# load it again
with open('/home/mckc/linear_model.pkl', 'rb') as fid:
    Net = cPickle.load(fid)
    
map = np.load('/home/mckc/map.npy')

In [3]:
while True:
    # Capture frame-by-frame
    #time.sleep(1)
    ret, frame = video_capture.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(frame.astype(np.uint8), 1)
    # Draw a rectangle around the faces
    if len(faces)>0:    
        for a,b in enumerate(faces):
            fac = np.array(gray)[b.top():b.bottom(),b.left():b.right()]       
            X,X_normal = Pre_Process(fac)
            Probability = Net.predict_proba(X.reshape(-1,9216))
            prob = np.amax(Probability)
            index = np.argmax(Probability)
            cv2.rectangle(frame, (b.left(), b.bottom()), (b.right(), b.top()), (255, 255, 255), 1)
            cv2.putText(frame,str(map[index])+' '+str(round(prob*100,2) )+'%',(b.left(),b.top()), cv2.FONT_HERSHEY_DUPLEX,1,(255,255,255), 1,2)            
            #scipy.misc.toimage(fac).save(str(uuid.uuid4()) +'.jpg')

    # Display the resulting frame
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        


# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()
